# Rellenar NaN de variables categóricas

Usando dataset ../../Data/02ParaLimpiar/02desastres_paralimpiar.csv

## Observaciones importantes

Al ser nuestro conjunto de datos sobre desastres naturales y al estar ciertas columnas representando atributos relacionados con desastres específicos, es posible que algunas columnas tengan valores faltantes debido a la ausencia de un desastre o la inexistencia de ciertos atributos en ciertos casos. En tales escenarios, puede ser apropiado tratar esos valores faltantes como significativos e introducir una nueva categoría para representar la ausencia o la naturaleza indefinida del atributo.

### Propuestas para manejar estos casos y representar los valores faltantes como indefinidos:

+ Identificar las columnas categóricas que contienen valores faltantes que representan la ausencia o la naturaleza indefinida de ciertos atributos.

+ Crear una nueva categoría o etiqueta para representar explícitamente los valores que faltan. Esta nueva categoría se puede denominar "Undefined" o "Sin definir", "Not Applicable" o "No aplicable", "No Data" o "Sin datos" o cualquier otra etiqueta que transmita adecuadamente la ausencia o la naturaleza indefinida del atributo.

+ Reemplazar los valores faltantes en las columnas categóricas con la nueva categoría o etiqueta. Usar el método fillna() de pandas para realizar este reemplazo.

### Criterio de borrado o relleno de columnas y valores de filas:

* Determinar con qué tipo de valores rellenar los faltantes de cada columna
* Asegurarse de que el tipo de dato rellenado es acorde a las interdependencias de columnas
* Probar cómo responde dataset tras fillna de valores faltantes
* Cuidado con reindexar columnas por separado

## Importar librerías

In [5]:
# pip install pyjanitor matplotlib missingno numpy pandas pyreadr seaborn session-info upsetplot

In [6]:
import missingno as msno
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

## Cargar datos

In [7]:
df = pd.read_csv('../../Data/02ParaLimpiar/02desastres_paralimpiar.csv', encoding='utf-8', delimiter=';', engine='python')

# Verificar carga de dataset

##  Resumen básico shape, info

In [8]:
df.shape
# Resultado: 16636 filas y 50 columnas)

(16636, 27)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16636 entries, 0 to 16635
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Dis No             16636 non-null  object 
 1   Year               16636 non-null  int64  
 2   Seq                16636 non-null  int64  
 3   Disaster Subgroup  16636 non-null  object 
 4   Disaster Type      16636 non-null  object 
 5   Disaster Subtype   13313 non-null  object 
 6   Country            16636 non-null  object 
 7   ISO                16636 non-null  object 
 8   Region             16636 non-null  object 
 9   Continent          16636 non-null  object 
 10  Location           14825 non-null  object 
 11  Origin             4085 non-null   object 
 12  Associated Dis     3593 non-null   object 
 13  Associated Dis2    763 non-null    object 
 14  OFDA Response      1716 non-null   object 
 15  Appeal             2559 non-null   object 
 16  Declaration        334

# Exploración de valores faltantes

## Tabulación de valores faltantes

In [10]:
df.isna()

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,...,AID Contribution,Dis Mag Value,Dis Mag Scale,Latitude,Longitude,Start Year,Start Month,End Year,End Month,CPI
0,False,False,False,False,False,False,False,False,False,False,...,True,True,False,True,True,False,True,False,True,False
1,False,False,False,False,False,False,False,False,False,False,...,True,True,False,True,True,False,True,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16631,False,False,False,False,False,False,False,False,False,False,...,True,True,False,True,True,False,False,False,False,True
16632,False,False,False,False,False,False,False,False,False,False,...,True,True,False,True,True,False,False,False,False,True
16633,False,False,False,False,False,False,False,False,False,False,...,True,True,False,True,True,False,False,False,False,True
16634,False,False,False,False,False,False,False,False,False,False,...,True,True,False,True,True,False,False,False,False,True


In [11]:
df.isna().sum()

Dis No                   0
Year                     0
Seq                      0
Disaster Subgroup        0
Disaster Type            0
Disaster Subtype      3323
Country                  0
ISO                      0
Region                   0
Continent                0
Location              1811
Origin               12551
Associated Dis       13043
Associated Dis2      15873
OFDA Response        14920
Appeal               14077
Declaration          13293
AID Contribution     15860
Dis Mag Value        11572
Dis Mag Scale         1220
Latitude             13861
Longitude            13861
Start Year               0
Start Month            395
End Year                 0
End Month              700
CPI                    106
dtype: int64

### Columnas categóricas con valores faltantes

In [12]:
# Columnas categóricas
df.select_dtypes(include=object).columns.to_list() # columns.to_list muestra solo los títulos de las cols

['Dis No',
 'Disaster Subgroup',
 'Disaster Type',
 'Disaster Subtype',
 'Country',
 'ISO',
 'Region',
 'Continent',
 'Location',
 'Origin',
 'Associated Dis',
 'Associated Dis2',
 'OFDA Response',
 'Appeal',
 'Declaration',
 'Dis Mag Scale',
 'Latitude',
 'Longitude',
 'CPI']

## Definición de variable categorical_columns

In [13]:
categorical_columns = ['Disaster Subtype', 'Location', 'Origin', 'Associated Dis', 'Associated Dis2', 'OFDA Response', 'Appeal', 'Declaration', 'AID Contribution', 'Dis Mag Scale', 'CPI']

In [14]:
categorical_data = df[categorical_columns]

## Calcular porcentaje de valores faltantes

In [16]:
missingness_percentage = categorical_data.isnull().mean() * 100

In [18]:
missingness_percentage_df = pd.DataFrame({'Porcentaje de valores faltantes': missingness_percentage})
missingness_percentage_df.sort_values(by='Porcentaje de valores faltantes', ascending=False, inplace=True)
print(missingness_percentage_df)

                  Porcentaje de valores faltantes
Associated Dis2                         95.413561
AID Contribution                        95.335417
OFDA Response                           89.685020
Appeal                                  84.617697
Declaration                             79.905025
Associated Dis                          78.402260
Origin                                  75.444818
Disaster Subtype                        19.974754
Location                                10.886030
Dis Mag Scale                            7.333494
CPI                                      0.637172


#### Recomendable borrar las columnas críticas que tengan porcentaje de valores faltantes muy altos (por encima de 70%) si no se pueden rellenar con datos lógicos en función de la interdependencia entre columnas:
Associated Dis2                         95.413561
AID Contribution                        95.335417
OFDA Response                           89.685020
Appeal                                  84.617697
Declaration                             79.905025
Associated Dis                          78.402260
Origin                                  75.444818

# Machine learning para rellenar NaN con predicciones

## SimpleImputer
https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

sklearn.impute.SimpleImputer

class sklearn.impute.SimpleImputer(*, missing_values=nan, strategy='mean', fill_value=None, verbose='deprecated', copy=True, add_indicator=False, keep_empty_features=False)

#### Definición de SimpleImputer

Imputador univariante para completar valores faltantes con estrategias simples.

Reemplaza los valores faltantes usando una estadística descriptiva (por ejemplo, media, mediana o más frecuente) a lo largo de cada columna, o usando un valor constante.

### Elegir estrategia de imputación

Según el porcentaje de faltantes y conocimiento del tema, seleccionar una estrategia adecuada para completar o reemplazar los valores faltantes en cada columna categórica.

Elección de estrategia: En este caso, se podría considerar 'most_frequent' o 'constante' para reemplazar los valores faltantes con el valor más frecuente en cada columna.

### Aplicar la estrategia de imputación elegida:

#### SimpleImputer con estrategia 'most_frequent'

In [ ]:
imputer = SimpleImputer(strategy='most_frequent')  # Reemplazar con la estrategia elegida, si no es 'most_frequent'
imputed_data = pd.DataFrame(imputer.fit_transform(categorical_data), columns=categorical_columns)

#### SimpleImputer con estrategia 'constant'

In [ ]:
imputer = SimpleImputer(strategy='constant')  # Reemplazar con la estrategia elegida, si no es constant
imputed_data = pd.DataFrame(imputer.fit_transform(categorical_data), columns=categorical_columns)

### Validar datos imputados

Examinar los datos categóricos imputados para verificar que se hayan reemplazado los valores faltantes. Realizar más análisis o modelado utilizando el conjunto de datos imputado.

### Otras técnicas de imputación disponibles en scikit-learn:

RandomForestClassifier, KNNImputer o IterativeImputer, según requisitos específicos.

# Guardar dataset en csv

In [15]:
# df.to_csv('../../Data/02ParaLimpiar/02desastres_fillna.csv', index=False, sep=';', encoding='utf-8')

Dataset con valores rellenados y/o filas vacías borradas.